In [1]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 131)

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Anteriormente separamos quais os exemplos que tinham a maior probabilidade de nos trazer ganho caso anotassemos os 'y' deles, mario fez o processo de labeling que nem da outra vez, no google shets, colocando 0 e 1 manualmente, analisando o titulo.

Vamos ver o resultado do active learning, como a gente mede o impacto ?

In [2]:
df1 = pd.read_csv("raw_data_with_labels.csv", index_col=0)
df1 = df1[df1['y'].notnull()]
df1.shape

(498, 16)

In [3]:
df2 = pd.read_csv("active_labels1_done.csv", index_col=0)
df2 = df2[df2['y'].notnull()]
df2['novo'] = 1
df2.shape

# 498 feitas inicialmente, e agora df2, as novas com 100 label novas pra gente usar.
# adicionamos a coluna novo, vamos entender logo mais abaixo

(100, 18)

In [4]:
df2.head(1)

,watch-title,y,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0,p,novo
505,Platform Overview - Machine Learning,0,4.298 visualizações,Publicado em 21 de mai. de 2019,Ciência e tecnologia,Platform Overview - Machine Learning,Google Cloud Platform\n\n\n\n\n\n\n\n\n\n\n\n\...,4.298 visualizações\n\n\n\n\n\n\n\n141\n\nGost...,https://i.ytimg.com/vi/QR_LQQ-vvko/maxresdefau...,1280.0,720.0,"In this short GCP Essentials video, see how GC...",1280.0,720.0,Alexis Moussine Pouchkine,/channel/UCJS9pqu9BzkAMNTmzNMNhvg,0.502,1


Antes de avaliar, mario ficou curioso de algo.

Se usarmos essas probabilidades que outros modelo nos deu, agora que temos as labels no dataset, quais serão as minhas métricas nesse dataset que acabamos de fazer as labels se usarmos as probabilidades lá do primeiro modelo.

O que a gente pode tentar enteder é, ta perto ou longe do que esperavamos.

In [5]:
# Nos dados de validação que temos, a average_precision era 0.19, agora 0.20, quer dizer que modelo ta realmente generalizando
# aqui o auc era 0.58, aqui ta bem mais baixo, mas ele ta muito sensivel ao numero de exemplos. Realmente esperto uma grande variação
# mas eles não estão absurdamente ruins ou diferentes do que esperavamos.

from sklearn.metrics import roc_auc_score, average_precision_score
average_precision_score(df2['y'], df2['p']), roc_auc_score(df2['y'], df2['p'])

(0.2037344613689981, 0.5386250885896527)

In [6]:
#juntamos df1 e df2, tirando a coluna p.
# quando juntarmos, a gente vair criar a coluna novo, que esta em df2 somente, tbm em df1, so que vai preenhcer com NaN

df = pd.concat([df1, df2.drop("p", axis=1)])

/home/jarbasjr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [7]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['watch-title']
# agora colocamos a coluna novo, e ao inves de ter os valores nulos, preenchemos com 0
# se o exemplo pertencer ao df novo ele vai ter 1, se pertencer aos exemplos que ja tinha as anotações, ele vai ter 0
# isso vai ajudar a comparar o efeito do active learning
df_limpo['novo'] = df['novo'].fillna(0)

### Parte padrão dos outros

In [8]:
clean_date = df['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)
#clean_date[1] = clean_date[1].map(lambda x: x[0].upper()+x[1:])

mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
clean_date.head()
df_limpo['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

In [9]:
views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo['views'] = views

In [10]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [11]:
features['tempo_desde_pub'] = (pd.to_datetime("2019-12-03") - df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features = features.drop(['tempo_desde_pub'], axis=1)

In [12]:
features.head()

,views,views_por_dia
0,28028,61.464912
1,1131,2.960733
2,1816,8.446512
3,1171,10.455357
4,1228,3.336957


## Aumenta validação

Antes a gente simplesmente dividia entre antes de abril e depois de abril, umas das maneiras que vamos usar pra entender se o active learning ta trazendo algum efeito pra gente é aumentar apenas o dataset de validação.


Geralmente preferimos que o set de validação seja o mesmo, mas nesse caso como tinhamos poucos dados de validação, e como to vendo que o auc ta muito instavel, quero ver se só adicionamos os novos exemplos ao dataset de validação, qual vai ser a minha estimativa das métricas, se eu usar os mesmo dados de treino(sem as linhas novas) mas validar nesse dataset aumentado.

No active learning, em geral, pensamos em add linhas no treino mas nesse caso estamos precisando de exemplos na validação.

In [13]:
mask_train = (df_limpo['date'] < "2019-04-01") & (df_limpo['novo'] == 0)

mask_val = (df_limpo['date'] >= "2019-04-01")

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((228, 2), (316, 2), (228,), (316,))

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

In [15]:
title_bow_train.shape

(228, 193)

In [16]:
from scipy.sparse import hstack, vstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [17]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((228, 195), (316, 195))

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [20]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=1000, n_jobs=6, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)

In [21]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [22]:
from sklearn.metrics import roc_auc_score, average_precision_score

Aqui está, antes tinhamos 0.19 de average_precision e agora 0.1872, apesar de estar melhor mario nao acha que piorou, mas temos poucos dados entao esta em um intervalo de variação.

O auc que esta sendo a metrica mais variavel, mais dificil de fazer se confiavel, aumentou um pouco, mas nao consideramos que melhorou mas ta dentro da variação esperada.

In [23]:
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.1872802752830275, 0.5935436218282933)

ANTES  
ap 0.1918043901336543, auc 0.5848024316109421 - mindf=2  

## AUMENTANDO TREINO

Maneira mais tradicional de fazer isso.

Dessa vez aumento os dados de treino e mantenho a mesma validação que eu tava usando antes de ter esses novos exemplso.

In [27]:
mask_train = (df_limpo['date'] < "2019-04-01")
mask_val = (df_limpo['date'] >= "2019-04-01") & (df_limpo['novo'] == 0)
#mask_val = (df_limpo['date'] >= "2019-04-01")

In [28]:
Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])


mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

p = mdl.predict_proba(Xval_wtitle)[:, 1]

average_precision_score(yval, p), roc_auc_score(yval, p)

(0.17697903398635972, 0.5964133738601823)

ANTES  
ap 0.1918043901336543, auc 0.5848024316109421 - mindf=2  

Valid nova, treino antigo  
ap 0.1872802752830275, auc 0.5935436218282933

Valid antiga, treino novo  
ap 0.17697903398635972, auc 0.5964133738601823

## Agora tudo novo, sem colocar todos os dados novos no treino ou na validação, mas simplesmente dividindo treino e validação em abril de 2019.

In [29]:
mask_train = (df_limpo['date'] < "2019-04-01")
#mask_val = (df_limpo['date'] >= "2019-04-01") & (df_limpo['novo'] == 0)
mask_val = (df_limpo['date'] >= "2019-04-01")

In [30]:
Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2)
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])


mdl = RandomForestClassifier(n_estimators=1000, random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

p = mdl.predict_proba(Xval_wtitle)[:, 1]

average_precision_score(yval, p), roc_auc_score(yval, p)

(0.19342838733336654, 0.6095324991310394)

ANTES  
ap 0.1918043901336543, auc 0.5848024316109421 - mindf=2  

Valid nova, treino antigo  
ap 0.1872802752830275, auc 0.5935436218282933

Valid antiga, treino novo  
ap 0.17697903398635972, auc 0.5964133738601823

Tudo novo  
ap 0.19342838733336654, auc 0.6095324991310394

Mario acha que isso ta dentro do esperado.


Claro que gostamos de ver mais subindo as métricas, mas nesse caso nao tem como comparamos muito com os valores antigos, pq mudamos tanto os dados de treino como de validação, e a gente nao pode realmente falar que o modelo ta muito melhor que o anterior.

Essa nao e a unica maneira de fazer o active learning, tem outras estrategias que a gente pode usar, queria te mostrar esse processo que faz muito sentido e mario ja viu funcionar muito bem em varios casos.

Não em dados tao pequenos, mas em casos com dataset de 6mil exemplos, e depois a empresa mandou pra algum serviço de anotação aqueles exemplos que pareciam confudir mais o modelo, foram aumentando aos poucos o dataset e a gente conseguiu otimizar o orçamento pra fazer essas anotações.



### obs: como a gente ta precisando muito de dados e temos apenas cerca de 1100~1200 exemplos, mario vai fazer as labels pra todos manualmente no google sheets,  pra que possamos realmente criar um modelo mais legal.

Em regra se a gente tiver mais dados tanto pra treinar como pra validar é melhor, claro que existem exceções como em time series, que as vezes temos muitos dados do passado e nao funciona tao bem quanto ter so de mais recentes. Mas se tiver que escolher entre ter mais dados e ter menos dados é melhor ter mais e ver que eles nao tao ajudando do que não ter mais dados.